In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class NN(nn.Module):
    def __init__(self, inputs=4, h1=8, h2=9, outputs=3):
        super().__init__()
        self.fc1 = nn.Linear(inputs,h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, outputs)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

In [4]:
model = NN()
model

NN(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [5]:
iris = load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [6]:
X = df.drop('target',axis=1).values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.parameters

<bound method Module.parameters of NN(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)>

In [8]:
epochs = 3
losses = []

for i in range(epochs):
    i+=1
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)

    print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  1  loss: 1.07153106
epoch:  2  loss: 1.04196560
epoch:  3  loss: 1.01462114


In [9]:
with torch.no_grad():
    y_val = model.forward(X_test)
    loss = criterion(y_val, y_test)
print(f'{loss:.8f}')

0.93779618


In [10]:
correct = 0
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_val = model.forward(data)
        print(f'{i+1:2}. {str(y_val):38}  {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1
print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')

 1. tensor([-0.9422, -0.8822, -0.5618])     1
 2. tensor([-0.9732, -0.9044, -0.6094])     1
 3. tensor([-0.1927, -0.3550, -0.3440])     0
 4. tensor([-1.0597, -0.9597, -0.5907])     1
 5. tensor([-1.3423, -1.1554, -0.6133])     2
 6. tensor([-1.5300, -1.3300, -0.7684])     2
 7. tensor([-0.2520, -0.4017, -0.3884])     0
 8. tensor([-0.1513, -0.3371, -0.3623])     0
 9. tensor([-1.3249, -1.1641, -0.6854])     2
10. tensor([-1.4171, -1.2380, -0.7375])     2
11. tensor([-1.4741, -1.2794, -0.7472])     2
12. tensor([-0.1501, -0.3250, -0.2962])     0
13. tensor([-1.4043, -1.2421, -0.7528])     2
14. tensor([-1.0802, -0.9789, -0.5977])     1
15. tensor([-1.2585, -1.1221, -0.6947])     2
16. tensor([-0.9528, -0.8840, -0.5743])     1
17. tensor([-1.1485, -1.0388, -0.6332])     2
18. tensor([-0.1622, -0.3416, -0.3672])     0
19. tensor([-1.0760, -0.9908, -0.6531])     1
20. tensor([-1.4371, -1.2406, -0.7311])     2
21. tensor([-0.1719, -0.3361, -0.3243])     0
22. tensor([-0.1942, -0.3698, -0.4